In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_5084/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/20 13:31:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/20 13:31:10 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Flights table
flights_data = {
    'flight_id': [1, 2, 3],
    'capacity': [2, 2, 1]
}

# Sample data for Passengers table
passengers_data = {
    'passenger_id': [101, 102, 103, 104, 105, 106, 107],
    'flight_id': [1, 1, 1, 2, 2, 3, 3],
    'booking_time': ['2023-07-10 16:30:00', '2023-07-10 17:45:00', '2023-07-10 12:00:00',
                     '2023-07-05 13:23:00', '2023-07-05 09:00:00', '2023-07-08 11:10:00',
                     '2023-07-08 09:10:00']
}

# Create DataFrame for Flights
flights_df = pd.DataFrame(flights_data)

# Create DataFrame for Passengers
passengers_df = pd.DataFrame(passengers_data)

# Convert booking_time to datetime type
passengers_df['booking_time'] = pd.to_datetime(passengers_df['booking_time'])

# Display the DataFrames
print("Flights table:")
print(flights_df)
print("\nPassengers table:")
print(passengers_df)


df_person = spark.createDataFrame(flights_df)
df_person.createOrReplaceTempView("Flights")

df_person = spark.createDataFrame(passengers_df)
df_person.createOrReplaceTempView("Passengers")



Flights table:
   flight_id  capacity
0          1         2
1          2         2
2          3         1

Passengers table:
   passenger_id  flight_id        booking_time
0           101          1 2023-07-10 16:30:00
1           102          1 2023-07-10 17:45:00
2           103          1 2023-07-10 12:00:00
3           104          2 2023-07-05 13:23:00
4           105          2 2023-07-05 09:00:00
5           106          3 2023-07-08 11:10:00
6           107          3 2023-07-08 09:10:00


In [12]:
query = """
    select
        p.passenger_id,
        if(f.capacity >= row_number() over(partition by p.flight_id order by p.booking_time asc), 'Comfirmed', 'Waitlist') as Status
    from Passengers p left join Flights f on p.flight_id = f.flight_id
    order by p.passenger_id asc
"""

In [13]:
spark.sql(query).show()

+------------+---------+
|passenger_id|   Status|
+------------+---------+
|         101|Comfirmed|
|         102| Waitlist|
|         103|Comfirmed|
|         104|Comfirmed|
|         105|Comfirmed|
|         106| Waitlist|
|         107|Comfirmed|
+------------+---------+

